In [10]:
# Class: Order with:
# @property for total
# @classmethod to load orders from orders.csv
# @staticmethod to convert currency
# Decorator @apply_tax(rate) to increase total
# Decorator @log_order(file) to log to file
# Chain decorators (@apply_tax + @log_order)
# File reading/writing involved
import csv

def apply_tax(rate):
    def deco(fn):
        def wrapper(self): 
            return fn(self) * (1 + rate)
        return wrapper
    return deco

def log_order(file):
    def deco(fn):
        def wrapper(self):
            val = fn(self)
            with open(file, "a") as f:
                f.write(f"{self.order_id},{val:.2f}\n")
            return val
        return wrapper
    return deco

class Order:
    def __init__(self, order_id, items):
        self.order_id, self.items = order_id, items

    @property
    def total(self):
        return sum(p*q for p, q in self.items)

    @classmethod
    def from_csv(cls, file="orders.csv"):
        orders = {}
        with open(file) as f:
            for oid, p, q in csv.reader(f):
                orders.setdefault(oid, []).append((float(p), int(q)))
        return [cls(oid, items) for oid, items in orders.items()]

    @staticmethod
    def convert_currency(amount, rate):
        return amount * rate

    @log_order("orders.log")
    @apply_tax(0.1)  
    def final_total(self):
        return self.total

if __name__ == "__main__":
    with open("orders.csv", "w", newline="") as f:
        csv.writer(f).writerows([[1,10,2],[2,20,1]])

    for o in Order.from_csv():
        print(o.order_id, "base=", o.total, "final=", o.final_total())
    print("100 USD in EUR =", Order.convert_currency(100, 0.92))


1 base= 20.0 final= 22.0
2 base= 20.0 final= 22.0
100 USD in EUR = 92.0
